In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

train = pd.read_csv('/content/drive/MyDrive/bda/trainby.csv')
train

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
10881,2012-12-19 19:00:00,4,0,1,1,15.58,19.695,50,26.0027,7,329,336
10882,2012-12-19 20:00:00,4,0,1,1,14.76,17.425,57,15.0013,10,231,241
10883,2012-12-19 21:00:00,4,0,1,1,13.94,15.910,61,15.0013,4,164,168
10884,2012-12-19 22:00:00,4,0,1,1,13.94,17.425,61,6.0032,12,117,129


In [ ]:
train['date']=train['datetime'].apply(lambda x:x.split()[0])
train['year']=train['datetime'].apply(lambda x:x.split()[0].split('-')[0])
train['month']=train['datetime'].apply(lambda x:x.split()[0].split('-')[1])
train['day']=train['datetime'].apply(lambda x:x.split()[0].split('-')[2])
train['hour']=train['datetime'].apply(lambda x:x.split()[1].split(':')[0])
train['minute']=train['datetime'].apply(lambda x:x.split()[1].split(':')[1])
train['second']=train['datetime'].apply(lambda x:x.split()[1].split(':')[2])

In [ ]:
from datetime import datetime
import calendar

In [ ]:
train['weekday']=train['date'].apply(lambda dateString:calendar.day_name[datetime.strptime(dateString,"%Y-%m-%d").weekday()])

In [ ]:
train = train[train['weather']!=4]

In [ ]:
# weekday 제외
drop_features = ['casual', 'registered', 'datetime', 'date', 'month', 'windspeed']

In [ ]:
train_fin = train.drop(drop_features, axis = 1)
train_fin

,season,holiday,workingday,weather,temp,atemp,humidity,count,year,day,hour,minute,second,weekday
0,1,0,0,1,9.84,14.395,81,16,2011,01,00,00,00,Saturday
1,1,0,0,1,9.02,13.635,80,40,2011,01,01,00,00,Saturday
2,1,0,0,1,9.02,13.635,80,32,2011,01,02,00,00,Saturday
3,1,0,0,1,9.84,14.395,75,13,2011,01,03,00,00,Saturday
4,1,0,0,1,9.84,14.395,75,1,2011,01,04,00,00,Saturday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10881,4,0,1,1,15.58,19.695,50,336,2012,19,19,00,00,Wednesday
10882,4,0,1,1,14.76,17.425,57,241,2012,19,20,00,00,Wednesday
10883,4,0,1,1,13.94,15.910,61,168,2012,19,21,00,00,Wednesday
10884,4,0,1,1,13.94,17.425,61,129,2012,19,22,00,00,Wednesday


In [ ]:
# weekday 데이터형태 변환
train_fin['weekday'] = train_fin['weekday'].map({'Saturday' : 6,
                                                 'Sunday' : 7,
                                                 'Monday' : 1,
                                                 'Tuesday' : 2,
                                                 'Wednesday' : 3,
                                                 'Thursday' : 4,
                                                 'Friday' : 5})

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_fin_X = train_fin[['season','holiday','workingday','weather','temp','atemp', 'weekday']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_fin_X, train_fin['count'] ,test_size = 0.3, random_state = 111)

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [ ]:
pred_train = model.predict(X_train)
pred_train

array([251.1042688 , 210.04227788, 119.32872531, ..., 197.21360894,
       215.05966642, 135.14945602])

In [ ]:
pred_test = model.predict(X_test)
pred_test

array([351.63721731, 210.0555041 , 130.27317892, ..., 248.20226108,
       113.88085549, 141.81837708])

In [ ]:
# weekday 포함한 모델

from sklearn.metrics import mean_squared_error

mse_train = mean_squared_error(y_true = y_train, y_pred = model.predict(X_train))
mse_test = mean_squared_error(y_true = y_test, y_pred = model.predict(X_test))

print('훈련데이터 mse :', mse_train)
print('테스트데이터 mse :', mse_test)

훈련데이터 mse : 26714.300119220956
테스트데이터 mse : 28286.396908152892
